In [1]:
import sys; sys.path.append("../../../../automl/")

In [14]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model.xgboost import XGBClassification
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

In [15]:
create_ml_data_dir()
configure_root_logger()

## Constants

In [16]:
RANDOM_SEED = 77
DATA_PATH = Path("../../../../data/")
CONFIG_PATH = Path("../../../../configs/config.yaml")
N_JOBS = 16

In [17]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

In [18]:
#cfg["stack_features"] = ["lamau_814_full_dataset", "lamann_autoint_8053_full_dataset"]

## Data

In [19]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_oof.parquet")
#df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [20]:
df_train["target"].value_counts(normalize=True)

0    0.940982
1    0.059018
Name: target, dtype: float64

In [21]:
# undersample the 0 class
#df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)


In [22]:
cat_columns = df_train.drop(columns=["target", "id"]).select_dtypes(int).columns.values.tolist()

In [23]:
X_train, y_train = df_train[cfg["selected_features"] + cfg["stack_features"] + cat_columns], df_train["target"]
#X_test, y_test = df_test[cfg["selected_features"] + cat_columns], df_test["target"]

In [24]:
display(y_train.value_counts(normalize=True))
#display(y_test.value_counts(normalize=True))

0    0.940982
1    0.059018
Name: target, dtype: float64

In [25]:
#categorical_features = ohe_cols# + oe_cols

## Model

In [26]:
metric = RocAuc()

In [15]:
# save_std_out = sys.stdout
# sys.stdout = open("stdout_logs", "w")

In [28]:
model = XGBClassification(n_jobs=16, time_series=True)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-11 07:35:50,653] - [   START    ] - Tuning XGBClassification
[2024-11-11 07:36:18,474] - [   OPTUNA   ] - Trial 0. New best score 0.8119761464109547 with parameters {'max_depth': 6, 'grow_policy': 'depthwise', 'max_leaves': 311, 'gamma': 3.1203728088487304, 'subsample': 0.2403950683025824, 'colsample_bytree': 0.15227525095137953, 'colsample_bylevel': 0.8795585311974417, 'reg_lambda': 6.011150117432088, 'reg_alpha': 7.080725777960454, 'min_child_weight': 0, 'class_weight': None, 'n_estimators': 319}
[2024-11-11 07:40:45,192] - [   OPTUNA   ] - Trial 3. New best score 0.8129923209545863 with parameters {'max_depth': 8, 'grow_policy': 'lossguide', 'max_leaves': 27, 'gamma': 18.186408041575643, 'subsample': 0.3329019834400152, 'colsample_bytree': 0.6962700559185838, 'colsample_bylevel': 0.3805399684804699, 'reg_lambda': 5.200680211778108, 'reg_alpha': 5.4671027934327965, 'min_child_weight': 3, 'class_weight': None, 'n_estimators': 444}
[2024-11-11 07:48:08,649] - [   OPTUNA   ] - 

In [33]:
pd.DataFrame().assign(imp=model.models[0].feature_importances_, names=model.models[0].feature_names_in_).sort_values("imp", ascending=False)

,imp,names
51,0.124526,lgb_8122_full_dataset
53,0.121289,xgb_81325_full_dataset
50,0.116655,lamau_81425_full_dataset
54,0.075602,lama_81298_full_dataset
52,0.059440,cb_8114_full_dataset
...,...,...
16,0.000000,feature_44
49,0.000000,feature_183
65,0.000000,feature_163
66,0.000000,feature_167


In [34]:
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cfg["stack_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(f'xgb_stack_time_series.csv', index=False)

In [17]:
print(metric(y_train, oof))

0.8140217779655352


In [19]:
model.model.get_feature_scores()

,Feature,Importance
0,lamau_814_full_dataset,155900.645331
1,feature_83,1862.207562
2,feature_14,1780.640471
3,feature_24,1777.973118
4,feature_148,1773.221087
...,...,...
59,feature_122,20.285800
60,feature_179,19.531767
61,feature_156,17.388200
62,feature_109,16.693633


In [20]:
MODEL_NAME = "lamau_stack_8140_full_dataset"
MODEL_DIR = Path(f"../../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [21]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
#joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
test["target"] = model.predict(test[cfg["selected_features"]+ cfg["stack_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

In [24]:
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cfg["stack_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(f'{MODEL_NAME}.csv', index=False)

In [26]:
model.model.get_feature_scores()[:50]

,Feature,Importance
0,lamau_814_full_dataset,78048.337071
1,xgb_81325_full_dataset,29297.001751
2,lama_81298_full_dataset,24098.444188
3,feature_162,1398.695796
4,feature_24,1321.674997
5,feature_18,1298.269506
6,feature_26,1290.120701
7,feature_145,1287.683197
8,feature_78,1279.916003
9,feature_36,1248.758602


In [24]:
imp = pd.DataFrame().assign(names=model.models[0].feature_names_, imp=model.models[0].feature_importances_)

In [41]:
imp.sort_values(by="imp", ascending=False).reset_index(drop=True).query("names == 'feature_185'")

,names,imp
61,feature_185,0.0


In [29]:
cat_columns

['feature_7',
 'feature_31',
 'feature_60',
 'feature_61',
 'feature_71',
 'feature_109',
 'feature_122',
 'feature_156',
 'feature_163',
 'feature_167',
 'feature_179',
 'feature_185']

## With Time series cross val

In [13]:
metric = RocAuc()

In [14]:
df_train = df_train.sort_values(by="id").reset_index(drop=True)
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]

In [15]:
model = LightGBMClassification(n_jobs=16, time_series=True)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 08:27:33,388] - [   START    ] - Tuning LightGBMClassification
[2024-11-07 08:27:40,118] - [   OPTUNA   ] - Trial 0. New best score 0.7903405446081995 with parameters {'max_depth': 6, 'num_leaves': 488, 'min_data_in_leaf': 188, 'bagging_fraction': 0.7993292420985183, 'bagging_freq': 0, 'feature_fraction': 0.49359671220172163, 'lambda_l1': 0.5808361216819946, 'lambda_l2': 8.661761457749352, 'min_gain_to_split': 12.022300234864176, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:03,043] - [   OPTUNA   ] - Trial 2. New best score 0.7913515589848906 with parameters {'max_depth': 5, 'num_leaves': 194, 'min_data_in_leaf': 117, 'bagging_fraction': 0.8925879806965068, 'bagging_freq': 0, 'feature_fraction': 0.708540663048167, 'lambda_l1': 5.924145688620425, 'lambda_l2': 0.46450412719997725, 'min_gain_to_split': 12.150897038028766, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:12,225] - [   OPTUNA   ] - Trial 3. New best score 0.8048686053278628 with param

In [27]:
none_oofs_idx = oof[np.any(np.isnan(oof), axis=1)].shape[0]

In [32]:
metric(y_train[none_oofs_idx:], oof[none_oofs_idx:])

0.8095227594190041

In [34]:
MODEL_NAME = "lgb_8095_full_dataset_time_series"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [35]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[none_oofs_idx:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

## TEST 
**81.22112399468679**

## Inference

In [27]:
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv('lgb_813.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method